In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pygold.graphs import configurar_grafica
from pygold.download import descargar_datos_yahoo

warnings.filterwarnings("ignore")
configurar_grafica()

In [2]:
# Ejemplo de uso:
ticker_symbol = "GC=F"  # oro
start_date = "2012-01-01"

# Get today date in format 'YYYY-MM-DD'
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")

# end_date = "2023-12-31"
interval = "1d"

data = descargar_datos_yahoo(ticker_symbol, start_date, end_date, interval)
if data is not None:
    print(data.head())

[*********************100%***********************]  1 of 1 completed

                   open         high          low        close    adj close  \
time                                                                          
2012-01-03  1568.900024  1604.300049  1566.000000  1599.699951  1599.699951   
2012-01-04  1596.599976  1618.599976  1594.500000  1611.900024  1611.900024   
2012-01-05  1610.800049  1624.099976  1597.800049  1619.400024  1619.400024   
2012-01-06  1622.500000  1627.800049  1616.099976  1616.099976  1616.099976   
2012-01-09  1614.300049  1620.099976  1607.500000  1607.500000  1607.500000   

            volume  
time                
2012-01-03     229  
2012-01-04     172  
2012-01-05     362  
2012-01-06      20  
2012-01-09       8  


In [10]:
data.loc["2023":"2024"]

,open,high,low,close,adj close,volume
time,,,,,,
2023-01-03,1836.199951,1839.699951,1836.199951,1839.699951,1839.699951,29
2023-01-04,1845.599976,1859.099976,1845.599976,1852.800049,1852.800049,25
2023-01-05,1855.199951,1855.199951,1834.800049,1834.800049,1834.800049,24
2023-01-06,1838.400024,1868.199951,1835.300049,1864.199951,1864.199951,26
2023-01-09,1867.000000,1880.000000,1867.000000,1872.699951,1872.699951,62
...,...,...,...,...,...,...
2024-09-16,2580.399902,2580.399902,2580.399902,2580.399902,2580.399902,28
2024-09-17,2581.199951,2581.800049,2564.300049,2564.300049,2564.300049,22
2024-09-18,2570.699951,2570.699951,2549.199951,2570.699951,2570.699951,25


In [11]:
calcular_tasa_rendimiento(data.loc["2023":"2024"])

np.float64(0.23652053657807848)

In [6]:
# Crear media móvil simple de 30 días
data["SMA fast"] = data["close"].rolling(30).mean()

# Crear media móvil simple de 60 días
data["SMA slow"] = data["close"].rolling(60).mean()

In [ ]:
# Plot the results
data[["close", "SMA fast", "SMA slow"]].plot(figsize=(15, 8))

In [ ]:
# Plot the results
data[["close", "SMA fast", "SMA slow"]].loc["2024"].plot(figsize=(15, 8))

In [10]:
data["position"] = np.nan

# Create the condition
data.loc[(data["SMA fast"] > data["SMA slow"]), "position"] = 1
data.loc[(data["SMA fast"] < data["SMA slow"]), "position"] = -1

In [ ]:
# Representamos toda la señal para asegurarnos de que sea correcta

year = "2024"

# Seleccionar toda la señal en una lista de índices para representar sólo estos puntos
idx_open = data.loc[data["position"] == 1].loc[year].index
idx_close = data.loc[data["position"] == -1].loc[year].index


# Adaptar el tamaño del gráfico
plt.figure(figsize=(15, 6))

# Representar los puntos de la señal larga 'open' en verde y la 'sell' en rojo
plt.scatter(
    idx_open, data.loc[idx_open]["close"].loc[year], color="#57CE95", marker="^"
)
plt.scatter(idx_close, data.loc[idx_close]["close"].loc[year], color="red", marker="v")


# Representar la resistencia para asegurarse de que las condiciones se completan
plt.plot(data["close"].loc[year].index, data["close"].loc[year], alpha=0.35)

plt.plot(data["close"].loc[year].index, data["SMA fast"].loc[year], alpha=0.35)

plt.plot(data["close"].loc[year].index, data["SMA slow"].loc[year], alpha=0.35)


# Mostrar el gráfico
plt.show()

In [ ]:
# Calcular el porcentaje de variación del activo
data["pct"] = data["close"].pct_change(1)

# Calcular la rentabilidad (retorno) de la estrategia
data["return"] = data["pct"] * data["position"].shift(1)


data["return"].plot(figsize=(15, 8))

In [ ]:
data["return"].cumsum().plot(figsize=(15, 8))

In [14]:
return_serie = data["adj close"].pct_change(1).dropna()
return_serie.name = "return"

In [ ]:
return_serie

In [ ]:
# Calcular el índice de Sortino
mean = np.mean(return_serie)  # 252
vol = np.std(return_serie[return_serie < 0])  # np.sqrt(252)
sortino = np.sqrt(252) * mean / vol

print(f"Sortino: {'%.3f' % sortino}")

In [ ]:
# Necesitamos calcular la covarianza entre el mercado y la cartera
sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)
sp500.name = "SP500"

# Los concatenamos para hacer las covarianzas
val = pd.concat((return_serie, sp500), axis=1).dropna()
val

In [ ]:
# Calculamos la matriz de covarianzas
cov_var_mat = np.cov(val.values, rowvar=False)
cov_var_mat

In [ ]:
# Calculamos beta
cov = cov_var_mat[0][1]
var = cov_var_mat[1][1]

beta = cov / var

print(f"Beta: {'%.3f' % beta}")

In [ ]:
# Calculamos alpha
alpha = (252 * mean * (1 - beta)) * 100
print(f"Alpha: {'%.1f' % alpha} %")

In [19]:
def drawdown_function(serie):
    # Calculamos la suma de los rendimientos
    cum = serie.dropna().cumsum() + 1

    # Calculamos el máximo de la suma en el período (máximo acumulado) # (1,3,5,3,1) --> (1,3,5,5,5)
    running_max = np.maximum.accumulate(cum)

    # Calculamos el drawdown
    drawdown = cum / running_max - 1
    return drawdown

In [ ]:
# Calculamos el drawdown
drawdown = drawdown_function(return_serie)
drawdown

In [ ]:
# Adaptar el tamaño de la figura
plt.figure(figsize=(15, 8))

# Representar el drawdown
plt.fill_between(
    drawdown.index, drawdown * 100, 0, drawdown, color="#CE5757", alpha=0.65
)

# Añadimos el título
plt.title("Drawdown")

# Etiqueta del eje de la y
plt.ylabel("Drawdown en %")

plt.show()

In [ ]:
# Computation Max drawdown
max_drawdown = -np.min(drawdown) * 100
print(f"Max drawdown: {'%.1f' % max_drawdown} %")

In [28]:
def BackTest(serie, annualiazed_scalar=252):
    # Importar el benchmark
    sp500 = yf.download("^GSPC")["Adj Close"].pct_change(1)

    # Cambiar el nombre
    sp500.name = "SP500"

    # Concatenar los retornos y el sp500
    val = pd.concat((serie, sp500), axis=1).dropna()
    # Calcular el drawdown
    drawdown = drawdown_function(serie) * 100

    # Calcular el max drawdown
    max_drawdown = -np.min(drawdown)

    # Put a subplots
    fig, (cum, dra) = plt.subplots(1, 2, figsize=(20, 6))

    # Put a Suptitle
    fig.suptitle("Backtesting", size=20)

    # Returns cumsum chart
    cum.plot(serie.cumsum() * 100, color="#39B3C7")

    # SP500 cumsum chart
    cum.plot(val["SP500"].cumsum() * 100, color="#B85A0F")

    # Put a legend
    cum.legend(["Portfolio", "SP500"])

    # Set individual title
    cum.set_title("Cumulative Return", size=13)

    cum.set_ylabel("Cumulative Return %", size=11)

    # Put the drawdown
    dra.fill_between(drawdown.index, 0, drawdown, color="#C73954", alpha=0.65)

    # Set individual title
    dra.set_title("Drawdown", size=13)

    dra.set_ylabel("drawdown en %", size=11)

    # Plot the graph
    plt.show()

    # Calcular el índice sortino
    sortino = np.sqrt(annualiazed_scalar) * serie.mean() / serie.loc[serie < 0].std()

    # Calcular el índice  beta
    beta = np.cov(val[["return", "SP500"]].values, rowvar=False)[0][1] / np.var(
        val["SP500"].values
    )

    # Calcular el índice  alpha
    alpha = annualiazed_scalar * (serie.mean() - beta * serie.mean())

    # Imprimir los estadísticos
    print(f"Sortino: {np.round(sortino,3)}")
    print(f"Beta: {np.round(beta,3)}")
    print(f"Alpha: {np.round(alpha*100,3)} %")
    print(f"MaxDrawdown: {np.round(max_drawdown,3)} %")

In [ ]:
BackTest(return_serie, annualiazed_scalar=252)